## Getting Started
Load the data from the CSV. The columns that hold JSON data are **genres** and **keywords**, so turn those into JSON objects.

In [3]:
import pandas as pd
from json import loads

df = pd.read_csv("tmdb_5000_movies.csv")
df["genres"] = df["genres"].apply(loads)
df["keywords"] = df["keywords"].apply(loads)

## Data Exploration
Here's what a single row of the dataset looks like.

In [2]:
df.head(1)

,budget,genres,homepage,id,keywords,original_language,original_title,overview,popularity,production_companies,production_countries,release_date,revenue,runtime,spoken_languages,status,tagline,title,vote_average,vote_count
0,237000000,"[{'id': 28, 'name': 'Action'}, {'id': 12, 'nam...",http://www.avatarmovie.com/,19995,"[{'id': 1463, 'name': 'culture clash'}, {'id':...",en,Avatar,"In the 22nd century, a paraplegic Marine is di...",150.437577,"[{""name"": ""Ingenious Film Partners"", ""id"": 289...","[{""iso_3166_1"": ""US"", ""name"": ""United States o...",2009-12-10,2787965087,162.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}, {""iso...",Released,Enter the World of Pandora.,Avatar,7.2,11800


Each row has an **id**, **genres**, and **keywords**. We need to extract the genres and keywords into their own CSVs.

In [3]:
df[["id", "genres", "keywords"]].head(5)

,id,genres,keywords
0,19995,"[{'id': 28, 'name': 'Action'}, {'id': 12, 'nam...","[{'id': 1463, 'name': 'culture clash'}, {'id':..."
1,285,"[{'id': 12, 'name': 'Adventure'}, {'id': 14, '...","[{'id': 270, 'name': 'ocean'}, {'id': 726, 'na..."
2,206647,"[{'id': 28, 'name': 'Action'}, {'id': 12, 'nam...","[{'id': 470, 'name': 'spy'}, {'id': 818, 'name..."
3,49026,"[{'id': 28, 'name': 'Action'}, {'id': 80, 'nam...","[{'id': 849, 'name': 'dc comics'}, {'id': 853,..."
4,49529,"[{'id': 28, 'name': 'Action'}, {'id': 12, 'nam...","[{'id': 818, 'name': 'based on novel'}, {'id':..."


### Example of the genres schema
Each genre has an "id" and "name" attribute that we can extract.

In [4]:
df["genres"][0]

[{'id': 28, 'name': 'Action'},
 {'id': 12, 'name': 'Adventure'},
 {'id': 14, 'name': 'Fantasy'},
 {'id': 878, 'name': 'Science Fiction'}]

### Example of the keywords schema
The keywords schema is exactly like the genres schema, so this means we can reuse some code.

In [5]:
df["keywords"][0]

[{'id': 1463, 'name': 'culture clash'},
 {'id': 2964, 'name': 'future'},
 {'id': 3386, 'name': 'space war'},
 {'id': 3388, 'name': 'space colony'},
 {'id': 3679, 'name': 'society'},
 {'id': 3801, 'name': 'space travel'},
 {'id': 9685, 'name': 'futuristic'},
 {'id': 9840, 'name': 'romance'},
 {'id': 9882, 'name': 'space'},
 {'id': 9951, 'name': 'alien'},
 {'id': 10148, 'name': 'tribe'},
 {'id': 10158, 'name': 'alien planet'},
 {'id': 10987, 'name': 'cgi'},
 {'id': 11399, 'name': 'marine'},
 {'id': 13065, 'name': 'soldier'},
 {'id': 14643, 'name': 'battle'},
 {'id': 14720, 'name': 'love affair'},
 {'id': 165431, 'name': 'anti war'},
 {'id': 193554, 'name': 'power relations'},
 {'id': 206690, 'name': 'mind and soul'},
 {'id': 209714, 'name': '3d'}]

## Data Preprocessing

In [6]:
genre_relations = [(row["id"], genre["id"]) for _, row in df.iterrows() for genre in row["genres"]]
genre_relations[:10] # truncate preview because there's a lot

[(19995, 28),
 (19995, 12),
 (19995, 14),
 (19995, 878),
 (285, 12),
 (285, 14),
 (285, 28),
 (206647, 28),
 (206647, 12),
 (206647, 80)]

In [7]:
genres = {(genre["id"], genre["name"]) for row in df["genres"] for genre in row}
genres

{(12, 'Adventure'),
 (14, 'Fantasy'),
 (16, 'Animation'),
 (18, 'Drama'),
 (27, 'Horror'),
 (28, 'Action'),
 (35, 'Comedy'),
 (36, 'History'),
 (37, 'Western'),
 (53, 'Thriller'),
 (80, 'Crime'),
 (99, 'Documentary'),
 (878, 'Science Fiction'),
 (9648, 'Mystery'),
 (10402, 'Music'),
 (10749, 'Romance'),
 (10751, 'Family'),
 (10752, 'War'),
 (10769, 'Foreign'),
 (10770, 'TV Movie')}

In [8]:
keyword_relations = [(row["id"], keyword["id"]) for _, row in df.iterrows() for keyword in row["keywords"]]
keyword_relations[:10] # truncate preview because there's a lot

[(19995, 1463),
 (19995, 2964),
 (19995, 3386),
 (19995, 3388),
 (19995, 3679),
 (19995, 3801),
 (19995, 9685),
 (19995, 9840),
 (19995, 9882),
 (19995, 9951)]

In [9]:
keywords = {(keyword["id"], keyword["name"]) for row in df["keywords"] for keyword in row}
list(keywords)[:10] # truncate preview because there's a lot

[(217992, 'thoughts of retirement'),
 (175343, 'garage sale'),
 (159330, 'empty safe'),
 (210689, 'virility'),
 (5922, 'north carolina'),
 (212, 'london england'),
 (7579, 'rich woman - poor man'),
 (162842, 'stolen cars'),
 (165399, 'editing'),
 (1774, 'news broadcast')]

In [14]:
columns = ["id", "title", "popularity", "vote_average", "vote_count"]
movies = [tuple(x) for x in df[columns].values]
movies[:10]

[(19995, 'Avatar', 150.437577, 7.2, 11800),
 (285, "Pirates of the Caribbean: At World's End", 139.082615, 6.9, 4500),
 (206647, 'Spectre', 107.37678799999999, 6.3, 4466),
 (49026, 'The Dark Knight Rises', 112.31295, 7.6, 9106),
 (49529, 'John Carter', 43.926995, 6.1, 2124),
 (559, 'Spider-Man 3', 115.69981399999999, 5.9, 3576),
 (38757, 'Tangled', 48.681969, 7.4, 3330),
 (99861, 'Avengers: Age of Ultron', 134.27922900000002, 7.3, 6767),
 (767, 'Harry Potter and the Half-Blood Prince', 98.885637, 7.4, 5293),
 (209112, 'Batman v Superman: Dawn of Justice', 155.790452, 5.7, 7004)]

## Output to CSV

In [16]:
import csv

In [17]:
def output(name, data, header):
    with open(name + '.csv', 'w') as f:
        writer = csv.writer(f, delimiter=',', lineterminator='\n')
        writer.writerow(header)
        for item in data:
            writer.writerow(item)

In [ ]:
output("genres", genres, ('id', 'name'))
output("keywords", keywords, ('id', 'name'))
output("genre_relations", genre_relations, ('movie_id', 'genre_id'))
output("keyword_relations", keyword_relations, ('movie_id', 'keyword_id'))

In [18]:
output("movies", movies, ('id', 'title', 'popularity', 'vote_average', 'vote_count'))